In [1]:
from preprocessing import get_model_dataset, create_train_test, min_max_scale, df_to_xy, read_file, lag_features
from lstm import create_model
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [4]:
first_year = 2019
last_year = 2021
file = f"./data/processed_data/{first_year}-{last_year}.csv"
df_read = read_file(file)

In [12]:
print("---Info----")
df_read.info()

---Info----
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5123798 entries, 0 to 5123797
Data columns (total 13 columns):
 #   Column           Dtype  
---  ------           -----  
 0   Unnamed: 0       int64  
 1   Quote_date       object 
 2   Expire_date      object 
 3   Underlying_last  float64
 4   Strike           float64
 5   Ask              float64
 6   Bid              float64
 7   Bid_strike       float64
 8   Ask_strike       float64
 9   Moneyness        float64
 10  Ttl              int64  
 11  Volatility       float64
 12  R                float64
dtypes: float64(9), int64(2), object(2)
memory usage: 508.2+ MB


In [11]:
print(df_read)

# Max Time to live
print("Longest time to live: ", df_read["Ttl"].max())

         Unnamed: 0  Quote_date Expire_date  Underlying_last  Strike      Ask  \
0           1354913  2019-01-02  2019-01-04          2509.98   800.0  1711.10   
1           1354914  2019-01-02  2019-01-04          2509.98   900.0  1611.40   
2           1354915  2019-01-02  2019-01-04          2509.98  1000.0  1511.40   
3           1354916  2019-01-02  2019-01-04          2509.98  1050.0  1462.19   
4           1354917  2019-01-02  2019-01-04          2509.98  1100.0  1412.20   
...             ...         ...         ...              ...     ...      ...   
5123793     6521988  2021-12-31  2024-12-20          4766.39  8400.0   300.00   
5123794     6521989  2021-12-31  2024-12-20          4766.39  8600.0   300.00   
5123795     6521990  2021-12-31  2024-12-20          4766.39  8800.0   300.90   
5123796     6521991  2021-12-31  2024-12-20          4766.39  9000.0   300.00   
5123797     6521992  2021-12-31  2024-12-20          4766.39  9200.0   300.00   

             Bid  Bid_strik

In [75]:
tests = [10000, 1586, 1089, 365*2, 365, 180, 90, 60, 30, 15, 0]
for test in tests:
    print('Count Ttl >', test, ':', df_read["Ttl"][df_read['Ttl'] > test].count())

Count Ttl > 10000 : 0
Count Ttl > 1586 : 0
Count Ttl > 1089 : 408
Count Ttl > 730 : 88853
Count Ttl > 365 : 362735
Count Ttl > 180 : 900201
Count Ttl > 90 : 1834026
Count Ttl > 60 : 2260711
Count Ttl > 30 : 3123010
Count Ttl > 15 : 4037921
Count Ttl > 0 : 5123798


In [76]:
tests = [50, 25, 10, 5, 2, 1.5, 1.03, 1, 0.97, 0.5, 0]
print(df_read["Moneyness"].max())
print(df_read["Moneyness"].min())
for test in tests:
    print('Moneyness Ttl >', test, ':', df_read["Moneyness"][df_read['Moneyness'] > test].count() / df_read["Moneyness"].count() * 100)

47.9294
0.4562775510204082
Moneyness Ttl > 50 : 0.0
Moneyness Ttl > 25 : 0.07223157509331946
Moneyness Ttl > 10 : 0.28633056962823283
Moneyness Ttl > 5 : 0.6770368386888007
Moneyness Ttl > 2 : 6.3422094313632185
Moneyness Ttl > 1.5 : 13.902499669190707
Moneyness Ttl > 1.03 : 61.69587481785972
Moneyness Ttl > 1 : 69.07145832056611
Moneyness Ttl > 0.97 : 76.7483417574229
Moneyness Ttl > 0.5 : 99.99865334269617
Moneyness Ttl > 0 : 100.0


In [77]:
print(df_read["Strike"][df_read["Ttl"] <= 3*365].max())
print(df_read["Strike"].min())
tester = "Strike"
tests = [8600, 7600, 6600, 5600, 4600, 3600, 2600, 1600, 600, 100, 0]
for test in tests:
    print(tester, '>', test, ':', df_read[tester][df_read[tester] > test].count() / df_read[tester].count() * 100)

9200.0
100.0
Strike > 8600 : 0.00046840254045924526
Strike > 7600 : 0.0021858785221431443
Strike > 6600 : 0.2711270038358265
Strike > 5600 : 1.4702765409565326
Strike > 4600 : 7.6027001845115665
Strike > 3600 : 30.2223467826015
Strike > 2600 : 73.19896295677543
Strike > 1600 : 94.19842468418935
Strike > 600 : 99.42501636481376
Strike > 100 : 99.92644128437537
Strike > 0 : 100.0


In [78]:
df_read[(df_read["Strike"] == 9200)]

,Unnamed: 0,Quote_date,Expire_date,Underlying_last,Strike,Ask,Bid,Bid_strike,Ask_strike,Moneyness,Ttl,Volatility,R
5058345,6456540,2021-12-21,2024-12-20,4648.98,9200.0,19.7,0.5,0.000054,0.002141,0.505324,1095,0.135275,0.96
5067505,6465700,2021-12-22,2024-12-20,4696.40,9200.0,300.0,0.0,0.000000,0.032609,0.510478,1094,0.136229,0.96
5076778,6474973,2021-12-23,2024-12-20,4725.38,9200.0,19.2,0.0,0.000000,0.002087,0.513628,1093,0.136510,0.97
5085894,6484089,2021-12-27,2024-12-20,4791.49,9200.0,19.2,0.0,0.000000,0.002087,0.520814,1089,0.138249,0.98
5095447,6493642,2021-12-28,2024-12-20,4787.26,9200.0,12.7,3.5,0.000380,0.001380,0.520354,1088,0.138268,0.99
5104935,6503130,2021-12-29,2024-12-20,4792.94,9200.0,19.2,0.0,0.000000,0.002087,0.520972,1087,0.138236,0.99
5114469,6512664,2021-12-30,2024-12-20,4779.28,9200.0,19.2,0.0,0.000000,0.002087,0.519487,1086,0.137746,0.98
5123797,6521992,2021-12-31,2024-12-20,4766.39,9200.0,300.0,0.0,0.000000,0.032609,0.518086,1085,0.136456,0.97


In [79]:
df_read[df_read["Quote_date"] <= "2021-12-31"].max()

Unnamed: 0            6521992
Quote_date         2021-12-31
Expire_date        2024-12-20
Underlying_last       4792.94
Strike                 9200.0
Ask                    4693.5
Bid                    4624.9
Bid_strike             46.249
Ask_strike             46.745
Moneyness             47.9294
Ttl                      1095
Volatility           0.543244
R                        2.62
dtype: object

In [80]:
df_read[(df_read["Strike"] == 9200)]

,Unnamed: 0,Quote_date,Expire_date,Underlying_last,Strike,Ask,Bid,Bid_strike,Ask_strike,Moneyness,Ttl,Volatility,R
5058345,6456540,2021-12-21,2024-12-20,4648.98,9200.0,19.7,0.5,0.000054,0.002141,0.505324,1095,0.135275,0.96
5067505,6465700,2021-12-22,2024-12-20,4696.40,9200.0,300.0,0.0,0.000000,0.032609,0.510478,1094,0.136229,0.96
5076778,6474973,2021-12-23,2024-12-20,4725.38,9200.0,19.2,0.0,0.000000,0.002087,0.513628,1093,0.136510,0.97
5085894,6484089,2021-12-27,2024-12-20,4791.49,9200.0,19.2,0.0,0.000000,0.002087,0.520814,1089,0.138249,0.98
5095447,6493642,2021-12-28,2024-12-20,4787.26,9200.0,12.7,3.5,0.000380,0.001380,0.520354,1088,0.138268,0.99
5104935,6503130,2021-12-29,2024-12-20,4792.94,9200.0,19.2,0.0,0.000000,0.002087,0.520972,1087,0.138236,0.99
5114469,6512664,2021-12-30,2024-12-20,4779.28,9200.0,19.2,0.0,0.000000,0.002087,0.519487,1086,0.137746,0.98
5123797,6521992,2021-12-31,2024-12-20,4766.39,9200.0,300.0,0.0,0.000000,0.032609,0.518086,1085,0.136456,0.97
